# Calculations

In [1]:
using PyPlot, Random, StatsBase, Distributions 
include("./evodi.jl"); using .evodi

In [2]:
# parameters
c = 0.1
s2M = 0.1
s2V = 0.01
kstar = .5
kappa_init = 0.25
K_values = [10^i for i in range(2,stop=3,length=10)]

T = 10000
N = 1000
tau = 500
Nstat = 25

## Figure 1

In [3]:
kf_values = [0,.5]
km_values = 0:0.1:.5
filename = "data_fig1A"
evodi.simu_Lambda_write(c, s2M, kf_values, km_values, T, N, Nstat, filename)

## Figure 2

In [4]:
K_range = [100, 300, 600]

Random.seed!(123);
results = Dict()
for (k,max_size) in enumerate(K_range)
    for i = 1:Nstat
        results[k,i] = evodi.run_dyn(c, kappa_init, s2M, s2V, N, T, tau, max_size=max_size)
    end
end

filename = "data_fig2ABC"
open("_Outputs/"*filename*".txt", "w") do f
    for (key,values) in results
        write(f,"$(key[1]) $(key[2])")
        for v in values
            write(f, " $v")
        end
        write(f, "\n")
    end
end

In [5]:
filename = "data_fig2D"
Random.seed!(123);
evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename)

In [6]:
qmale = 8
filename = "data_fig2E"
Random.seed!(123);
@time evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename, qmale=qmale)

In [7]:
qmale = 4
filename = "data_fig2F"
Random.seed!(123);
@time evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename, qmale=qmale)

## Figure 3

In [8]:
Random.seed!(123);
c_range = 0:0.02:.15
DL_values, DL_err = zeros(length(c_range)), zeros(length(c_range))
for (j,c) in enumerate(c_range)
    Lkmkmin,  Lkmkmin_err  = evodi.simu_Lambda(kstar, 0., s2M, c, N, T, Nstat)
    Lkmkstar, Lkmkstar_err = evodi.simu_Lambda(kstar, kstar, s2M, c, N, T, Nstat)
    DL_values[j], DL_err[j] = Lkmkmin - Lkmkstar, Lkmkmin_err + Lkmkstar_err
end

filename = "data_fig3A1"
open("_Outputs/"*filename*".txt", "w") do f
    for (i,c) in enumerate(c_range)
        write(f,"$(c) $(DL_values[i])  $(DL_err[i])\n")
    end
end

In [9]:
Random.seed!(123);
c_range = 0:0.02:.2

L_values, L_err = zeros(length(c_range)), zeros(length(c_range))
for (i,c) in enumerate(c_range)
    L_values[i], L_err[i] = evodi.simu_Lambda(kstar, 0., s2M, c, N, T, Nstat, chi_male=true)
end

filename = "data_fig3A2"
open("_Outputs/"*filename*".txt", "w") do f
    for (i,c) in enumerate(c_range)
        write(f,"$(c) $(L_values[i])  $(L_err[i])\n")
    end
end

## Figure S1

In [10]:
N_values = [500, 1000, 2000, 4000, 8000]

Random.seed!(123);
for N in N_values
    filename = "data_figS1_$(N)"
    evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename)
end

In [11]:
s2V_values = [0.005, 0.01, 0.02, 0.04, 0.08]

Random.seed!(123);
for s2V in s2V_values
    filename = "data_figS1_$(s2V)"
    evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename)
end

## Figure S2

In [12]:
s2e_values = [0., 0.1, 1.]
K_values = [10^i for i in range(2,stop=3,length=10)]
kappa_init = 0.25

Random.seed!(123);
for s2e in s2e_values
    filename = "data_figS2_$(s2e)"
   evodi.simu_kappa_write(c, s2M, s2V, K_values, N, T, tau, kappa_init, Nstat, filename, σ2e=s2e)
end

## Figure S3

In [13]:
function simu_kappa_write_mu(c, s2M, s2V, K, N, T, tau, kappa, Nstat, prog_mig_values, u_soft_values,
                                filename; qmale=Inf, σ2e=0., mig_before_sampling=false, mig_after_sampling=false)
    open("_Outputs/"*filename*".txt", "w") do f
        write(f,"c s2M s2V K m u N T kf km\n")
        for prob_mig in prog_mig_values
            for u_soft in u_soft_values
                for s in 1:Nstat
                    res = evodi.run_dyn(c, kappa, s2M, s2V, N, T, tau, max_size=K, qmale=qmale, σ2e=σ2e,
                            prob_mig=prob_mig, u_soft=u_soft,
                            mig_before_sampling=mig_before_sampling, mig_after_sampling=mig_after_sampling)
                    for n = 1:floor(Int, T/tau)
                        kf, km = round.(res[1:2,n+1], digits=4)
                        write(f,"$c $(s2M) $(s2V) $K $(prob_mig) $(u_soft) $N $(n*tau) $(kf) $(km)\n")
                    end
                end
            end
        end
    end
end

In [14]:
K = 100
u_soft_values = [10^i for i in range(-1,stop=0,length=10)]
prob_mig_values = [0]

filename = "data_figS3A"
res_kf, res_km = simu_kappa_read_usoft(filename, K, T, Nstat, u_soft_values)

In [15]:
K = 100
u_soft_values = [10^i for i in range(-1,stop=0,length=10)]
prob_mig_values = [0]

filename = "data_figS3A"
res_kf, res_km = simu_kappa_read_usoft(filename, K, T, Nstat, u_soft_values)

In [16]:
K = 100
u_soft_values = [0]
prob_mig_values = [10^i for i in range(-3,stop=-1,length=10)]

filename = "data_figS3B"
Random.seed!(123);
@time simu_kappa_write_mu(c, s2M, s2V, K, N, T, tau, kappa_init, Nstat, prob_mig_values, u_soft_values, 
                          filename, mig_after_sampling=true)

In [17]:
K = 100
u_soft_values = [10^i for i in range(-1,stop=0,length=5)]
prob_mig_values = [10^i for i in range(-3,stop=-1,length=5)]

filename = "data_fig3C"
Random.seed!(123);
@time simu_kappa_write_mu(c, s2M, s2V, K, N, T, tau, kappa_init, Nstat, prob_mig_values, u_soft_values, 
                          filename, mig_after_sampling=true)